Notebook is the exploration of the Vision Model from Deep SeekAI, A Chinese company lok

In [1]:
import torch
from transformers import AutoModelForCausalLM
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

In [2]:
model_path = "deepseek-ai/deepseek-vl-7b-chat"
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from transformers import BitsAndBytesConfig                                
                                 
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   # bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
vl_gpt = AutoModelForCausalLM.from_pretrained(model_path,
                                              trust_remote_code=True,
                                              quantization_config=nf4_config,
                                             device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Error issue
Faced the error: ImportError: cannot import name 'Mapping' from 'collections' 
Had to modify the import Mapping to be collections.abc module. This had to be done in multiple files

import collections.abc
for type_name in collections.abc.__all__:
    setattr(collections, type_name, getattr(collections.abc, type_name))

    
- aimachine/lib/python3.10/site-packages/attrdict/default.py

- aimachine/lib/python3.10/site-packages/attrdict/merge.py

- aimachine/lib/python3.10/site-packages/attrdict/mixins.py

- aimachine/lib/python3.10/site-packages/attrdict/mapping.py

In [4]:
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 11.73 GiB of which 23.88 MiB is free. Including non-PyTorch memory, this process has 11.68 GiB memory in use. Of the allocated memory 11.52 GiB is allocated by PyTorch, and 13.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>Describe each stage of this image.",
        "images": ["/home/aicoder/gitfolder/python_de_learners_data/code_script_notebooks/projects/huggingface_AWS/cutma_dspy_strictjson.png"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

In [6]:
vl_gpt.device

device(type='cuda', index=0)

In [6]:
pil_images = load_pil_images(conversation)

In [7]:
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

In [8]:
input_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

RuntimeError: Input type (c10::BFloat16) and bias type (c10::Half) should be the same

In [ ]:
outputs = vl_gpt.language_model.generate(
    input_embeds=input_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    user_cache=True
)